## Importing all the libraries

In [1]:
#Importing the libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
final = pd.read_csv("16thFeb2022_100k_120k_data_merged_with_sf_v2.csv")

In [3]:
final.drop(columns =['Unnamed: 0','code','l1_finish_time','l1_pc','DXCAT3',
                    'query_source','telehealth_encounter',
                    'DX Category','DX Context'],axis = 1,inplace = True)

In [4]:
final.shape

(2088529, 40)

In [5]:
for column in final:
    if final[column].dtype == 'float64':
        final[column] = pd.to_numeric(final[column], downcast = 'float')
    if final[column].dtype == 'int64':
        final[column] = pd.to_numeric(final[column], downcast = 'unsigned')

In [6]:
final.columns

Index(['code1', 'final_codes', 'DX Codetype', 'DX Condition Type', 'enc_id',
       'meat', 'text', 'page', 'section', 'NLP_VERSION',
       'parent_rxcui_mapping', 'final_code_family', 'dos_start', 'dos_end',
       'enc_class', 'unique_codes', 'code_count', 'npi.type', 'chart_id',
       'medicine_count', 'labtest_count',
       'embedded radiology/lab results_count', 'devices_count',
       'abnormal findings_count', 'generic term_count', 'plan_count',
       'procedure_count', 'radiology_count', 'labs_count', 'rx_count',
       'referral_count', 'symptoms_count', 'therapy_count', 'procedures_count',
       'others_count', 'physical_exam_count', 'evaluatory_term_count',
       'imaging_count', 'project_id', 'output'],
      dtype='object')

In [7]:
final[final['dos_start'].isna()].shape

(209, 40)

In [8]:
final.dropna(axis=0, subset=['dos_start','dos_end'],inplace = True)

In [9]:
for column in final:
    if final[column].dtype == 'float64':
        final[column] = pd.to_numeric(final[column], downcast = 'float')
    if final[column].dtype == 'int64':
        final[column] = pd.to_numeric(final[column], downcast = 'unsigned')

## Creating New Features

#### Features related to meat

In [10]:
final.meat.value_counts().head()

[]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [11]:
final["Has MEAT"] = np.where(final["meat"] != "[]",1,0)

In [12]:
final["Has MEAT"].value_counts()

1    1572738
0     515582
Name: Has MEAT, dtype: int64

In [13]:
final.columns

Index(['code1', 'final_codes', 'DX Codetype', 'DX Condition Type', 'enc_id',
       'meat', 'text', 'page', 'section', 'NLP_VERSION',
       'parent_rxcui_mapping', 'final_code_family', 'dos_start', 'dos_end',
       'enc_class', 'unique_codes', 'code_count', 'npi.type', 'chart_id',
       'medicine_count', 'labtest_count',
       'embedded radiology/lab results_count', 'devices_count',
       'abnormal findings_count', 'generic term_count', 'plan_count',
       'procedure_count', 'radiology_count', 'labs_count', 'rx_count',
       'referral_count', 'symptoms_count', 'therapy_count', 'procedures_count',
       'others_count', 'physical_exam_count', 'evaluatory_term_count',
       'imaging_count', 'project_id', 'output', 'Has MEAT'],
      dtype='object')

In [14]:
final["rx_count"].value_counts()

0      947306
1      151846
2      149066
3      120085
4      112248
        ...  
128         1
260         1
265         1
208         1
194         1
Name: rx_count, Length: 192, dtype: int64

In [15]:
final["Has RX MEAT"] = np.where((final["rx_count"] != 0) | (final["medicine_count"] != 0),1,0)

In [16]:
final["Has RX MEAT"].value_counts()

1    1366083
0     722237
Name: Has RX MEAT, dtype: int64

In [17]:
final["Has Procedure MEAT"] = np.where((final["procedure_count"] != 0) | (final["procedures_count"] != 0) ,1,0)

In [18]:
final["Has Labs MEAT"] = np.where((final["labs_count"] != 0) | (final["labtest_count"] != 0),1,0)

In [19]:
final["Has RX MEAT"].value_counts()

1    1366083
0     722237
Name: Has RX MEAT, dtype: int64

In [20]:
final["Has Labs MEAT"].value_counts()

0    1359708
1     728612
Name: Has Labs MEAT, dtype: int64

#### Other Features

In [21]:
number_of_encounters_per_chart = pd.DataFrame(final.groupby("chart_id").enc_id.nunique()).reset_index()

In [22]:
number_of_encounters_per_chart.columns = ["chart_id","number_of_encounters_per_chart"]

In [23]:
number_of_encounters_per_chart

,chart_id,number_of_encounters_per_chart
0,39358,21
1,39388,5
2,39400,2
3,393645,5
4,393649,4
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,1
19944,5382,3
19945,5387858,1
19946,539049,10


In [24]:
final = final.merge(number_of_encounters_per_chart, on = 'chart_id', how='left')

In [25]:
number_of_codes_per_chart = pd.DataFrame(final.groupby("chart_id").code1.nunique()).reset_index()

In [26]:
number_of_codes_per_chart.columns = ["chart_id","number_of_code1_per_chart"]

In [27]:
number_of_codes_per_chart

,chart_id,number_of_code1_per_chart
0,39358,2
1,39388,7
2,39400,3
3,393645,14
4,393649,11
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,2
19944,5382,6
19945,5387858,1
19946,539049,6


In [28]:
final = final.merge(number_of_codes_per_chart, on = 'chart_id', how='left')

In [29]:
number_of_occurences_of_code1_per_chart = pd.DataFrame(final.reset_index().groupby(["chart_id","code1"]).index.count()).reset_index()

In [30]:
number_of_occurences_of_code1_per_chart.head(10)

,chart_id,code1,index
0,39358,G809,186
1,39358,Z684,19
2,39388,I10,28
3,39388,I5030,3
4,39388,I509,56
5,39388,L98499,5
6,39388,N189,13
7,39388,R569,5
8,39388,Z684,5
9,39400,E119,2


In [31]:
number_of_occurences_of_code1_per_chart.columns = ["chart_id","code1","number_of_occurences_of_code1_per_chart"]

In [32]:
number_of_occurences_of_code1_per_chart.head(10)

,chart_id,code1,number_of_occurences_of_code1_per_chart
0,39358,G809,186
1,39358,Z684,19
2,39388,I10,28
3,39388,I5030,3
4,39388,I509,56
5,39388,L98499,5
6,39388,N189,13
7,39388,R569,5
8,39388,Z684,5
9,39400,E119,2


In [33]:
final = final.merge(number_of_occurences_of_code1_per_chart, on = ['chart_id','code1'], how='left')

In [34]:
final["DX incidence per encounter"] = final["number_of_occurences_of_code1_per_chart"]/final["number_of_encounters_per_chart"]

In [35]:
final["DX incidence per encounter"]

0          4.000000
1          4.000000
2          1.000000
3          4.000000
4          0.333333
             ...   
2088315    4.000000
2088316    4.000000
2088317    3.000000
2088318    4.000000
2088319    4.000000
Name: DX incidence per encounter, Length: 2088320, dtype: float64

In [36]:
number_of_encounters_per_chart_and_code1 = pd.DataFrame(final.groupby(["chart_id","code1"]).enc_id.nunique()).reset_index()

In [37]:
number_of_encounters_per_chart_and_code1.columns = ["chart_id","code1","number_of_encounters_per_chart_and_code1"]

In [38]:
number_of_encounters_per_chart_and_code1

,chart_id,code1,number_of_encounters_per_chart_and_code1
0,39358,G809,21
1,39358,Z684,19
2,39388,I10,5
3,39388,I5030,3
4,39388,I509,5
...,...,...,...
183776,5390815-1LR-2021-22,E162,1
183777,5390815-1LR-2021-22,I213,1
183778,5390815-1LR-2021-22,I214,1
183779,5390815-1LR-2021-22,I219,1


In [39]:
final = final.merge(number_of_encounters_per_chart_and_code1, on = ['chart_id','code1'], how='left')

In [40]:
number_of_pages_per_chart_per_enc = pd.DataFrame(final.groupby(["chart_id","enc_id"]).page.nunique()).reset_index()

In [41]:
number_of_pages_per_chart_per_enc.columns = ["chart_id","enc_id","number_of_pages_per_chart_per_enc"]

In [42]:
number_of_pages_per_chart_per_enc

,chart_id,enc_id,number_of_pages_per_chart_per_enc
0,39358,1,4
1,39358,2,3
2,39358,3,4
3,39358,4,3
4,39358,5,3
...,...,...,...
162909,5390815-1LR-2021-22,0,4
162910,5390815-1LR-2021-22,1,4
162911,5390815-1LR-2021-22,2,2
162912,5390815-1LR-2021-22,3,2


In [43]:
final = final.merge(number_of_pages_per_chart_per_enc, on = ['chart_id','enc_id'], how='left')

In [44]:
number_of_pages_in_a_chart = pd.DataFrame(final.groupby("chart_id").page.nunique()).reset_index()

In [45]:
number_of_pages_in_a_chart.columns = ["chart_id","number_of_pages_in_a_chart"]

In [46]:
number_of_pages_in_a_chart

,chart_id,number_of_pages_in_a_chart
0,39358,60
1,39388,19
2,39400,7
3,393645,26
4,393649,14
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,2
19944,5382,8
19945,5387858,2
19946,539049,18


In [47]:
final = final.merge(number_of_pages_in_a_chart, on = 'chart_id', how='left')

In [48]:
number_of_unique_sections_in_a_chart = pd.DataFrame(final.groupby("chart_id").section.nunique()).reset_index()

In [49]:
number_of_unique_sections_in_a_chart.columns = ["chart_id","number_of_unique_sections_in_a_chart"]

In [50]:
number_of_unique_sections_in_a_chart

,chart_id,number_of_unique_sections_in_a_chart
0,39358,8
1,39388,9
2,39400,5
3,393645,12
4,393649,4
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,4
19944,5382,7
19945,5387858,2
19946,539049,5


In [51]:
final = final.merge(number_of_unique_sections_in_a_chart, on = 'chart_id', how='left')

In [52]:
import ast

In [53]:
final["meat"] = final["meat"].apply(lambda s: ast.literal_eval(s) if type(s) != float else [])

In [54]:
final["number_of_meat_per_Dx"] = final["meat"].apply(lambda x: len(x))

In [55]:
final["final_codes"] = final["final_codes"].apply(lambda s: ast.literal_eval(s) if type(s) != float else [])

In [56]:
final["number_of_unique_codes_in_final_codes_per_Dx"] = final["final_codes"].apply(lambda x: len(set(x)))

In [57]:
final["parent_rxcui_mapping"] = final["parent_rxcui_mapping"].apply(lambda s: ast.literal_eval(s) if type(s) != float else [])

In [58]:
final["parent_rxcui_mapping"][0]

[6809]

In [59]:
final["number_of_unique_parent_rxcui_mapping_per_Dx"] = final["parent_rxcui_mapping"].apply(lambda x: len(set(x)))

In [60]:
final["code1_family"] = final["code1"].apply(lambda x: x[:3] if x else None)

In [61]:
number_of_code1_family_per_chart = pd.DataFrame(final.groupby("chart_id").code1_family.nunique()).reset_index()

In [62]:
number_of_code1_family_per_chart.columns = ["chart_id","number_of_code1_family_per_chart"]

In [63]:
number_of_code1_family_per_chart

,chart_id,number_of_code1_family_per_chart
0,39358,2
1,39388,6
2,39400,3
3,393645,11
4,393649,7
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,2
19944,5382,6
19945,5387858,1
19946,539049,6


In [64]:
final = final.merge(number_of_code1_family_per_chart, on = 'chart_id', how='left')

In [65]:
final["Acute Condition"] = np.where(final['DX Condition Type'] == "Acute",1,0)

In [66]:
number_of_acute_conditions_per_chart = pd.DataFrame(final.groupby("chart_id")["Acute Condition"].sum()).reset_index()

In [67]:
number_of_acute_conditions_per_chart.columns = ["chart_id","number_of_acute_conditions_per_chart"]

In [68]:
number_of_acute_conditions_per_chart

,chart_id,number_of_acute_conditions_per_chart
0,39358,0
1,39388,5
2,39400,12
3,393645,22
4,393649,66
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,0
19944,5382,8
19945,5387858,0
19946,539049,3


In [69]:
final = final.merge(number_of_acute_conditions_per_chart, on = 'chart_id', how='left')

In [70]:
final["Chronic Condition"] = np.where(final['DX Condition Type'] == "Chronic",1,0)

In [71]:
number_of_chronic_conditions_per_chart = pd.DataFrame(final.groupby("chart_id")["Chronic Condition"].sum()).reset_index()

In [72]:
number_of_chronic_conditions_per_chart.columns = ["chart_id","number_of_chronic_conditions_per_chart"]

In [73]:
number_of_chronic_conditions_per_chart

,chart_id,number_of_chronic_conditions_per_chart
0,39358,63
1,39388,97
2,39400,4
3,393645,162
4,393649,125
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,4
19944,5382,10
19945,5387858,4
19946,539049,18


In [74]:
final = final.merge(number_of_chronic_conditions_per_chart, on = 'chart_id', how='left')

#### Features realted to DOS created from dos_start

In [75]:
final['dos_start_year'] = pd.DatetimeIndex(final['dos_start']).year

In [76]:
final['dos_start'] = pd.to_datetime(final['dos_start'])

In [77]:
final['dos_start_quarter'] = pd.PeriodIndex(final['dos_start'], freq='Q')

In [78]:
number_of_unique_dos_per_chart = pd.DataFrame(final.groupby("chart_id").dos_start.nunique()).reset_index()

In [79]:
number_of_unique_dos_per_chart.columns = ["chart_id","number_of_unique_dos_start_per_chart"]

In [80]:
number_of_unique_dos_per_chart

,chart_id,number_of_unique_dos_start_per_chart
0,39358,21
1,39388,5
2,39400,1
3,393645,5
4,393649,4
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,1
19944,5382,3
19945,5387858,1
19946,539049,10


In [81]:
final = final.merge(number_of_unique_dos_per_chart, on = 'chart_id', how='left')

#### Dx occured in different years

In [82]:
dos_in_different_year = pd.DataFrame(final.groupby(['chart_id','code1']).dos_start_year.nunique().reset_index())

In [83]:
dos_in_different_year.columns = ['chart_id','code1','dos_count_of_different_year_per_chart_and_code1']

In [84]:
final = final.merge(dos_in_different_year, on = ['chart_id','code1'], how='left')

In [85]:
final["dos_in_different_year_per_chart_per_dx"] = np.where(final["dos_count_of_different_year_per_chart_and_code1"] >1,1,0)

#### Dx occured in different quarters

In [86]:
dos_in_different_quarter = pd.DataFrame(final.groupby(['chart_id','code1']).dos_start_quarter.nunique().reset_index())

In [87]:
dos_in_different_quarter.columns = ['chart_id','code1','dos_count_of_different_quarter_per_chart_and_code1']

In [88]:
final = final.merge(dos_in_different_quarter, on = ['chart_id','code1'], how='left')

In [89]:
final["dos_in_different_quarter_per_chart_per_dx"] = np.where(final["dos_count_of_different_quarter_per_chart_and_code1"] >1,1,0)

In [90]:
percentage_of_autocodable_codes_per_chart = pd.DataFrame(round(final.groupby("chart_id").output.sum()/final.groupby("chart_id").output.count(),2).reset_index())

In [91]:
percentage_of_autocodable_codes_per_chart.columns = ['chart_id','percentage_of_autocodable_codes_per_chart']

In [92]:
percentage_of_autocodable_codes_per_chart

,chart_id,percentage_of_autocodable_codes_per_chart
0,39358,0.91
1,39388,0.49
2,39400,0.75
3,393645,0.80
4,393649,0.72
...,...,...
19943,537BD86C-98D8-4B75-B0FE-AB5E1E351957,0.50
19944,5382,0.44
19945,5387858,1.00
19946,539049,0.78


In [93]:
final = final.merge(percentage_of_autocodable_codes_per_chart, on = 'chart_id', how='left')

In [104]:
#final.to_csv("19thFeb2022_100k_120k_data_merged_with_sf_and_new_features_v2.csv")

In [1]:
import os

In [5]:
os.remove("CIOX_RAHIX_IC20190521142219_591M7914320_124850035.json")